# Previsão de Probabilidade de Preços por Desvio Padrão

## 1. Importação das Bibliotecas e Configurações


In [17]:
%config IPCompleter.greedy=True
import yfinance as yf
import datetime
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots


## 2. Definição de Parâmetros

In [18]:
hoje = "2023-01-12"
inicio = "2022-01-12"


dias_venc = 26 #modificar colocando quantos dias faltam para o vencimento da opção ou quantos dias gostaria para o range da previsão de probabilidade


In [19]:
ativos = ["PETR4", "VALE3", "BOVA11", "MGLU3", "CIEL3", "CSNA3", "COGN3", "IRBR3", "VIIA3", "B3SA3", "BBAS3", "WEGE3", 
          "USIM5", "BRFS3", "BBDC4", "ITSA4", "ABEV3", "JBSS3", "PRIO3"] #pode incluir mais ativos
ativos = [i + '.SA' for i in ativos]


## 3. Baixando dados

In [20]:
df = yf.download(ativos, start = inicio, end = hoje)['Adj Close']
df

[*********************100%***********************]  19 of 19 completed


,ABEV3.SA,B3SA3.SA,BBAS3.SA,BBDC4.SA,BOVA11.SA,BRFS3.SA,CIEL3.SA,COGN3.SA,CSNA3.SA,IRBR3.SA,ITSA4.SA,JBSS3.SA,MGLU3.SA,PETR4.SA,PRIO3.SA,USIM5.SA,VALE3.SA,VIIA3.SA,WEGE3.SA
Date,,,,,,,,,,,,,,,,,,,
2022-01-12,14.135219,11.188937,27.256163,17.224394,101.910004,23.840000,1.927126,2.05,22.592722,3.63,7.872667,34.065464,6.31,17.368481,21.360001,15.597748,78.011925,4.11,29.690409
2022-01-13,14.239713,11.169663,27.695179,17.531666,101.550003,24.010000,1.917632,2.08,22.506752,3.53,8.092765,34.654575,6.09,17.719124,22.020000,15.531334,76.825775,3.99,28.590038
2022-01-14,14.344207,11.776814,28.405069,17.813328,103.070000,24.299999,1.917632,2.16,22.111292,3.54,8.185882,34.963150,6.33,18.379498,22.900000,15.607236,77.272865,3.97,30.161999
2022-01-17,14.059223,11.911736,28.461115,17.915760,102.580002,24.750000,2.012564,2.17,21.612671,3.48,8.188748,34.579762,6.12,18.408720,22.820000,15.398505,76.871399,3.90,29.818130
2022-01-18,13.850234,11.863550,28.713314,18.240101,102.849998,23.320000,1.946112,2.25,22.171473,3.35,8.248833,34.224426,5.89,18.490534,23.920000,15.692625,78.750984,3.75,30.004803
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-05,14.290000,12.300000,34.570000,14.082477,104.269997,8.380000,4.890000,2.05,15.630000,0.93,8.340000,22.100000,2.66,23.879999,34.970001,7.510000,90.900002,2.34,37.480000
2023-01-06,14.320000,12.590000,35.049999,14.476594,105.500000,8.640000,4.960000,2.09,16.270000,0.99,8.480000,22.209999,2.76,23.740000,35.180000,7.590000,92.339996,2.41,38.029999
2023-01-09,14.400000,12.420000,34.709999,14.660000,105.349998,8.690000,4.980000,2.11,16.340000,1.06,8.540000,22.879999,2.83,23.870001,36.009998,7.620000,92.440002,2.43,37.919998


## 4. Tratamento os dados

In [21]:
retornos = df.pct_change()
retornos = retornos.dropna()
data_ref = hoje
preco_ref = df.iloc[-1]

## 5. Calculo do desvio

In [22]:
media = retornos.mean()


In [23]:
desvio = retornos.std()


## 6. Previsão para periodo

In [24]:
medret = (((1+media)**(dias_venc))-1)


In [25]:
desvret = (desvio*np.sqrt(dias_venc))


## 7. Calculo dos preços de cada Desvio

In [26]:
desvio_05_p = round(preco_ref*(1+medret+0.5*desvret),2)
desvio_10_p = round(preco_ref*(1+medret+desvret),2)
desvio_15_p = round(preco_ref*(1+medret+1.5*desvret),2)
desvio_20_p = round(preco_ref*(1+medret+2*desvret),2)

desvio_05_n = round(preco_ref*(1+medret-0.5*desvret),2)
desvio_10_n = round(preco_ref*(1+medret-desvret),2)
desvio_15_n = round(preco_ref*(1+medret-1.5*desvret),2)
desvio_20_n = round(preco_ref*(1+medret-2*desvret),2)

## 8. Visualizando os valores dos desvios dos preços

In [27]:
desvios = pd.DataFrame([desvio_20_p, desvio_15_p, desvio_10_p, desvio_05_p, desvio_05_n, desvio_10_n, desvio_15_n, desvio_20_n])
desvios = desvios.rename({0: "2,0 desvios alta", 1: "1,5 desvios alta", 2: "1,0 desvios alta", 3: "0,5 desvios alta", 
                         7: "2,0 desvios baixa", 6: "1,5 desvios baixa", 5: "1,0 desvios baixa", 4: "0,5 desvios baixa"})
desvios

,ABEV3.SA,B3SA3.SA,BBAS3.SA,BBDC4.SA,BOVA11.SA,BRFS3.SA,CIEL3.SA,COGN3.SA,CSNA3.SA,IRBR3.SA,ITSA4.SA,JBSS3.SA,MGLU3.SA,PETR4.SA,PRIO3.SA,USIM5.SA,VALE3.SA,VIIA3.SA,WEGE3.SA
"2,0 desvios alta",16.62,17.74,44.67,18.59,125.77,10.73,7.32,3.09,22.98,1.65,10.30,26.17,4.50,31.74,53.73,9.72,118.65,3.78,49.00
"1,5 desvios alta",16.05,16.77,42.76,17.77,121.87,9.98,6.92,2.89,21.46,1.51,9.95,25.02,4.11,30.14,50.80,9.14,112.91,3.47,46.74
"1,0 desvios alta",15.49,15.80,40.85,16.95,117.96,9.22,6.52,2.69,19.95,1.37,9.61,23.87,3.71,28.55,47.87,8.56,107.18,3.16,44.47
"0,5 desvios alta",14.92,14.82,38.94,16.13,114.06,8.47,6.12,2.49,18.44,1.24,9.26,22.72,3.31,26.95,44.93,7.97,101.44,2.86,42.20
"0,5 desvios baixa",13.78,12.88,35.13,14.49,106.25,6.97,5.32,2.10,15.41,0.96,8.57,20.42,2.51,23.76,39.07,6.81,89.96,2.24,37.67
"1,0 desvios baixa",13.22,11.91,33.22,13.67,102.35,6.22,4.92,1.90,13.90,0.82,8.23,19.27,2.11,22.16,36.14,6.23,84.23,1.93,35.40
"1,5 desvios baixa",12.65,10.94,31.31,12.85,98.44,5.47,4.53,1.70,12.39,0.69,7.88,18.12,1.71,20.57,33.20,5.65,78.49,1.63,33.13
"2,0 desvios baixa",12.08,9.96,29.40,12.03,94.54,4.71,4.13,1.50,10.87,0.55,7.54,16.97,1.31,18.97,30.27,5.06,72.75,1.32,30.86


## 9. Visualização dos dados nos gráficos

In [16]:
hoje_real = datetime.date.today()
inicio_real = hoje_real - datetime.timedelta(days = 365) # pegar 1 ano de dados
hoje_real = str(hoje_real)
inicio_real = str(inicio_real)

ticker = "PRIO3" #Colocar a ação que quer ver o gráfico
ticker = ticker + '.SA'

ativo = yf.download(ticker, start = inicio_real, end = hoje_real)


fig = make_subplots(rows=1, cols=1)
fig.add_trace(go.Scatter(name=ticker + ' Ajustado', x=ativo.index, y=ativo['Adj Close']),row=1,col=1)
fig.add_trace(go.Candlestick(name=ticker, x=ativo.index, close=ativo['Close'], open=ativo['Open'],high=ativo['High'], low=ativo['Low']))
fig.add_hline(y=desvios.loc["0,5 desvios alta",ticker], line = {'color': '#ee5253', 'dash':'dot'}, 
              annotation_text=(f'+0,5 desvio -> {desvios.loc["0,5 desvios alta",ticker]}'), annotation_position="top left")
fig.add_hline(y=desvios.loc["1,0 desvios alta",ticker], line = {'color': '#ee5253', 'dash':'dot'}, 
              annotation_text=(f'+1,0 desvio -> {desvios.loc["1,0 desvios alta",ticker]}'), annotation_position="top left")
fig.add_hline(y=desvios.loc["1,5 desvios alta",ticker], line = {'color': '#ee5253', 'dash':'dot'}, 
              annotation_text=(f'+1,5 desvio -> {desvios.loc["1,5 desvios alta",ticker]}'), annotation_position="top left")
fig.add_hline(y=desvios.loc["2,0 desvios alta",ticker], line = {'color': '#ee5253', 'dash':'dot'}, 
              annotation_text=(f'+2,0 desvio -> {desvios.loc["2,0 desvios alta",ticker]}'), annotation_position="top left")

fig.add_hline(y=desvios.loc["0,5 desvios baixa",ticker], line = {'color': '#ee5253', 'dash':'dot'}, 
              annotation_text=(f'-0,5 desvio -> {desvios.loc["0,5 desvios baixa",ticker]}'), annotation_position="bottom left")
fig.add_hline(y=desvios.loc["1,0 desvios baixa",ticker], line = {'color': '#ee5253', 'dash':'dot'}, 
              annotation_text=(f'-1,0 desvio -> {desvios.loc["1,0 desvios baixa",ticker]}'), annotation_position="bottom left")
fig.add_hline(y=desvios.loc["1,5 desvios baixa",ticker], line = {'color': '#ee5253', 'dash':'dot'}, 
              annotation_text=(f'-1,5 desvio -> {desvios.loc["1,5 desvios baixa",ticker]}'), annotation_position="bottom left")
fig.add_hline(y=desvios.loc["2,0 desvios baixa",ticker], line = {'color': '#ee5253', 'dash':'dot'}, 
              annotation_text=(f'-2,0 desvio -> {desvios.loc["2,0 desvios baixa",ticker]}'), annotation_position="bottom left")


fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()



[*********************100%***********************]  1 of 1 completed
